# Creation of datasets

In this notebook, we solve the TOV equations for NS with a specific EOS composed of three regions: at low energy scales we use a known table of EOS (either SLy or AP4); for mass densities larger than twice the saturation density we implement an agnostic speed-of-sound parametrization; finally, for a pressure larger than (200 MeV)^4, we implement a vacuum energy phase transition. 

We also collect the data to create the inputs and outputs for the training/validation/testing process of our deep neural networks (provided in a different python notebook).

Finally, we implement a section to study the EOSs predicted by the neural networks, we solve the TOV equations (and the equation for the tidal deformability) and we compare the result with the expected ones.

We work in G=c=M_sun=1 units.

## Import packages

In [1]:
using DifferentialEquations, DelimitedFiles, Interpolations, Symbolics, Printf, Plots, Roots, Random, Distributions, CSV, LaTeXStrings, Images, Statistics

## Load data and units conventions

Let us load the data and some useful constants. We load the SLy and AP4 EOSs (appropriately rescaled); we also upload the random mass density and speed-of-sound values to create the c_s parametrization. We generate the training and validation datasets from the files "matrixrho2.dat" and "matrixcs2.dat"; the testing dataset is generated from "matrixrhoTest.dat" and "matrixcsTest.dat". Uncomment the appropriate datasets.

In [4]:
#LOADING DATA
sly = readdlm("Rescaledsly.dat", Float64) #p,ϵ,ρ
ap4 = readdlm("Rescaledap4.dat", Float64); #p,ϵ,ρ
#Data for generating training and validation datasets
#matrix_ρ = readdlm("matrixrho2.dat", Float64) #loading ρ coordinates for segments, 1000 eos
#matrix_cs = readdlm("matrixcs2.dat", Float64); #loading cs coordinates for segments, 1000 eos
#Data for generating testing dataset
matrix_ρ = readdlm("matrixrhoTest.dat", Float64) #loading ρ coordinates for segments, 100 eos
matrix_cs = readdlm("matrixcsTest.dat", Float64); #loading cs coordinates for segments, 100 eos

In [5]:
#CONSTANTS FOR UNITS
const cspeed = 2.997 * 10.0^10 #cm s^-1
const hbar = 6.582 * 10.0^-22 #Mev s
const MeV = 1.602 * 10.0^-6 #g cm^2 s^-1
const Kp = MeV/(hbar^3 * cspeed^3) #in g and cm
const Kρ = MeV/(hbar^3 * cspeed^5) #in g and cm
const fm = (10.0^-13)^-3 #fm^-3 in cm^-3
const n0 = 0.16 * fm #nuclear saturation density in cm^-3
const m = 1.675 * 10.0^-24 #neutron mass in g
const Msun = 1.988 * 10.0^33 #in g
const Gconst = 6.67 * 10.0^-11 * 10.0^6/10.0^3 #cm^3/g/s^2
const Lu = Msun^-1 * cspeed^2 * Gconst^-1 #1cm in G=c=Msun=1
const Pu = Msun^-1 / Lu^3 / cspeed^2
const ρu = Msun^-1 / Lu^3;
const KK = (2.997 * 10.0^5)^2/(6.67 * 10.0^-20 * 1.988 * 10.0^30);#rescaling for λ to check with GW

## Functions definition

We defined the functions to create the EOS and to solve the TOV and tidal deformability equations.

<u>Important note</u>: in the function "csInterp", we used an old interpolation for the first value of the speed of sound namely:

"segm_matrix = vcat(segm_matrix, [eos[k,3] p1])".

This choice is obsolete and is now commented. The new and correct values is given by:

"segm_matrix = vcat(segm_matrix, [eos[k,3] sqrt(p1)])",

marked as new implementation. The training/validation dataset had been created with the old implementation, and, to ensure that initial hypothesis were the same, we used the old implementation for the testing dataset as well.

If you intend to use this code, we recommend to use the new implementation for the creation of all datasets. 

In [5]:
# function to find (closest) position in array of certain value IT WORKS FOR INCREASING VALUE
function findpos(matrix,value,column)
    for b = 1:length(matrix[:,column])
        Δx1 = abs(matrix[b,column] - value)
        Δx2 = abs(matrix[b+1,column] - value)
        if matrix[b+1,column] >= value
            if Δx1 < Δx2
                return b
                break
            else
                return b+1
                break
            end
        end
    end
end;

In [6]:
#Evaluating derivative functions to find quantities at threshold
function derThr(eos, k)
    s = 0.0
    for j = 1:k
        Δp = eos[j+1,1] - eos[j,1]
        Δϵ = eos[j+1,2] - eos[j,2]
        s = Δp/Δϵ
    end
    return s
end

#creating an interpolated function for speed of sound modification
function csInterp(eos, k, p1, matr_ρ, matr_cs, ds)
    #define matrix to store (ρ,cs) for segments modification
    segm_matrix = Matrix{Float64}(undef,0,2) 
    #segm_matrix = vcat(segm_matrix, [eos[k,3] p1]) #old implementation
    segm_matrix = vcat(segm_matrix, [eos[k,3] sqrt(p1)]) #new implementation
    for j = 1:7
      segm_matrix = vcat(segm_matrix, [matr_ρ[ds,j] matr_cs[ds,j]])  
    end
    
    cs_interp = linear_interpolation(segm_matrix[:,1],segm_matrix[:,2])
end

#create high energy eos
function heEOS(he_eos::AbstractMatrix{T}, eos, k, cs_interp) where T
    Δρ = 10.0^-5
    M = div((10.0^-3 - eos[k,3]),10.0^-5)+div((10.0^-2 - 10.0^-3),10.0^-4)+
        div((ρfin - 10.0^-2),10.0^-3)    
    ρold = eos[k,3]
    ϵold = eos[k,2]
    pold = eos[k,1]
    
    for j = 1:M-1
        ρnew = ρold + Δρ
        ϵnew = ϵold + Δρ * (ϵold + pold)/ρold
        pnew = pold + cs_interp(ρold)^2 * Δρ * (ϵold + pold)/ρold #cs^2=dp/dϵ
        if ρnew > ρfin #break cycle if we reach last mass density
            break
        end
        he_eos = vcat(he_eos, [pnew ϵnew ρnew])
        ρold = ρnew #store new data in old variable for next loop
        ϵold = ϵnew
        pold = pnew
        if ρnew < 10.0^-3
            Δρ = 10.0^-5
        elseif ρnew < 10.0^-2
            Δρ = 10.0^-4
        elseif ρnew < 10^-1
            Δρ = 10.0^-3
        else 
            Δρ = 10.0^-2
        end
    end
    return he_eos
end

#merge two EOS
function mergeEOS(eos_matrix::AbstractMatrix{T}, he_eos, eos, k) where T
    N = k + length(he_eos[:,1])
    for j = 1:N
        if j <= k
            eos_matrix = vcat(eos_matrix, [eos[j,1] eos[j,2] eos[j,3]])
        else
            eos_matrix = vcat(eos_matrix, [he_eos[j-k,1] he_eos[j-k,2] he_eos[j-k,3]])
        end 
    end
    return eos_matrix
end

#BUILDING EOS before Λ transition
function build(eos_matrix::AbstractMatrix{T}, eos, matr_ρ, matr_cs, ds) where T
    #find quantities at transition threshold
    k = findpos(eos,ρt,3)
    
    #Evaluating derivative functions to find quantities at threshold
    p1 = derThr(eos, k)
    
    #creating an interpolated function for speed of sound modification
    cs_interp = csInterp(eos, k, p1, matr_ρ, matr_cs, ds)
       
    #define matrix to store new part of eos
    he_eos = Matrix{Float64}(undef,0,3)
    he_eos = heEOS(he_eos, eos, k, cs_interp)
        
    #merge the two eos at right threshold
    eos_matrix = mergeEOS(eos_matrix, he_eos, eos, k)
    return eos_matrix
end;

In [7]:
#define interpolated eos before and after transition
function eos_interp(x) 
    if x < pc
        ϵ_fluid(x)
    elseif x >= pc
        ϵ_fluid(x + Λ) + Λ
    end
end 
#define derivative of interpolated eos before and after transition
function eos_prime_interp(x) 
    if x < pc
        ϵ_prime(x)
    elseif x >= pc
        ϵ_prime(x + Λ)
    end
end;

In [8]:
function integrator(P0)
    #condition when defining surface and stop integration
    condition(u, r, integrator) =  u[3]/P0 < 10.0^-12 #10^-10
    affect!(integrator) = terminate!(integrator)
    cb = DiscreteCallback(condition, affect!)
    
    u0 = [f0, h0, P0, H0, β0]
    
    #probably must use specialization SciMLBase.FullSpecialize for cycles
    prob = ODEProblem(tov!, u0, rspan)
    
    sol = solve(prob, RK4(), callback = cb, dt = 0.0005, adaptive = false)
    return sol
end;

In [9]:
#Define the system of eqs to solve
function tov!(du, u, p, r) #u=f,h,P,H,β
    du[1] = (1.0 - u[1] - 8.0 * π * r^2 * eos_interp(u[3]) )/r
    du[2] = -(u[2] * (-1.0 + u[1] - 8.0 * π * r^2 * u[3]))/(r * u[1])
    du[3] = (-1.0 + u[1] - 8.0 * π * r^2 * u[3]) * (u[3] + 
        eos_interp(u[3]) ) /(2.0 * r * u[1])
    du[4] = u[5]
    du[5] = (u[4] * (-u[1]^3 + (1.0 +  8.0 * π * r^2 * u[3])^3 -
            u[1] * (1.0 + 8.0 * π * r^2 * u[3]) * (-3.0 + 60.0 * π * r^2 * u[3] + 
            20.0 * π * r^2 * eos_interp(u[3]) ) 
            + u[1]^2 * (-3.0 + 60.0 * π * r^2 * u[3] + 
            8.0 * π * r^3 * eos_prime_interp(u[3]) * (-1.0 + u[1] - #ϵ'[r]=dϵ/dp*p'=dϵ/dp*du[3]
            8.0 * π * r^2 * u[3]) * (u[3] + 
            eos_interp(u[3]) ) /(2.0 * r * u[1])+
            20.0 * π * r^2 * eos_interp(u[3]) )) + 
            r * u[1] * (-1 + u[1] - 8.0 * π * r^2 * u[3]) * (1.0 + u[1] + 
            4.0 * π * r^2 * u[3] -
            4.0 * π * r^2 * eos_interp(u[3]) ) * u[5])/(r^2 * u[1]^2 * (1.0 - 
            u[1] + 8.0 * π * r^2 * u[3]))
end;

In [10]:
function cycleTOV(DATA_matrix::AbstractMatrix{T}, P0, Pf) where T
    #determine how long the loop is
    if Pf > 10.0^-2
        N = floor((10.0^-4 - P0)/(2.5*10.0^-6) + (10.0^-3 - 10.0^-4)/(2.5*10.0^-5) +
        (10.0^-2 - 10.0^-3)/(2.5*10.0^-4))
    elseif Pf > 10.0^-3
        N = floor((10.0^-4 - P0)/(2.5*10.0^-6) + (10.0^-3 - 10.0^-4)/(2.5*10.0^-5) +
        (Pf - 10.0^-3)/(2.5*10.0^-4))
    elseif Pf > 10.0^-4
        N = floor((10.0^-4 - P0)/(2.5*10.0^-6) + (Pf - 10.0^-4)/(2.5*10.0^-5))
    elseif Pf > 10.0^-5
        N = floor((Pf - P0)/(2.5*10.0^-6))
    end

    for i=1:N
        sol = integrator(P0)
    
        Rs = sol.t[end] #radius
        M = Rs/2 * (1 - sol[1,end]) #mass
        y = Rs * sol[5,end]/sol[4,end]
        C = M / (Rs / Lu * 10^-5) #compactness
        #Love number k2
        k2 = (8.0 * (1.0 - 2.0 * C)^2 * C^5 * (2.0 + 2.0 * C * (-1.0 + y) -
            y))/(5.0 * (2.0 * C * (6.0 + C^2 * (26.0 - 22.0 * y) - 3.0 * y +
            4 * C^4 * (1.0 + y) + 3.0 * C * (-8.0 + 5.0 * y) + C^3 * (-4.0 + 6.0 * y)) +
            3.0 * (1.0 - 2.0 * C)^2 * (2.0 + 2.0 * C * (-1.0 + y) - y) * log(1 - 2*C)))
        λ = 2.0/3.0 * k2 * (Rs / Lu * 10^-5)^5 * KK^5/(M)^5 #tidal deformability resc
                
        #store values in matrix, create a new row each M-R loop
        DATA_matrix = vcat(DATA_matrix, [P0/ρu M Rs/Lu*10.0^-5 λ])
    
        #increment on P0
        if P0 < 10.0^-4       
            P0 = P0 + 2.5*10.0^-6
        elseif P0 < 10.0^-3
            P0 = P0 + 2.5*10.0^-5
        elseif P0 < 10.0^-2
            P0 = P0 + 2.5*10.0^-4
        else 
            P0 = P0 + 2.5*10.0^-3
        end
    end
    return DATA_matrix
end;

## Equations Integration

We integrate here the TOV and tidal deformability equations for all possible EOSs' combinations.

We save the results only for those EOSs that have a maximum mass compatible with PSR J0952-0607 which has a M_max=2.35+-0.17.

The results for the training and validation datasets are saved in the folder "TOVs", those for the testing dataset are saved in "TOVsTest".

Note: choose baseline EOS (AP4 or SLy).

In [16]:
const ρt = 2.0 * n0 * m * ρu #mass density when first transition happens
const ρfin = 12.0 * n0 * m * ρu
const pc = 200.0^4 * Kp * Pu #threshold for vacuum energy trans. corresponds to qcd scale
#define initial values for solving integration as global
const r0 = 10^-5
const a0 = 1.0
const f0 = 1.0
const h0 = 1.0
const H0 = a0 * r0^2
const β0 = 2. * a0 * r0
const rspan = (r0, 200)
#All possible values of Λ considered
Λarray = [-194.0^4 * Kp * Pu, -150.0^4 * Kp * Pu, -120.0^4 * Kp * Pu, -95.0^4 * Kp * Pu,
    -50.0^4 * Kp * Pu, 0., 50.0^4 * Kp * Pu, 95.0^4 * Kp * Pu, 120.0^4 * Kp * Pu,
    194.0^4 * Kp * Pu];

In [17]:
Λ = 0.
ϵ_fluid = zeros(0)
ϵ_prime(x::Any) = zeros(0)
#DATA_matrix = Matrix{Float64}(undef,0,4) 

for q = 1:length(Λarray)
    Λ = Λarray[q]
    @show Λ
    for j = 1:length(matrix_ρ[:,1])
        z = j
        @show j
    
        eos_matrix = Matrix{Float64}(undef,0,3) #define matrix to store complete eos
        eos_matrix = build(eos_matrix, sly, matrix_ρ, matrix_cs, z)
        
        #ϵ[p] before transition
        ϵ_fluid = linear_interpolation(eos_matrix[:,1],eos_matrix[:,2])
        #define interpolated dϵ[p]/dp
        ϵ_prime(x) = only(Interpolations.gradient(ϵ_fluid, x) );
        
        #INTEGRATOR
        P0 = 2.5 * 10^-5  #this will vary at each step of loop
        Pf = 0. #define it
        #Find what Pf is
        if Λ > 0 && eos_matrix[end,1] > pc
            Pf = eos_matrix[end,1] - Λ
        else
            Pf = eos_matrix[end,1]
        end
    
        #create empty matrix to store data in loop
        DATA_matrix = Matrix{Float64}(undef,0,4) 

        @time DATA_matrix = cycleTOV(DATA_matrix, P0, Pf)

        Mmax, iMax = findmax(DATA_matrix[:,2])
        
        if Λ == 0.0
            temp = 0
        else
            temp = floor(Int,abs(Λ/(Kp * Pu))^(1/4)/sign(Λ))
        end
        
        #Store TOV integration
        if 2.18 < Mmax < 2.52
            #store for training and validation dataset
            #dumping = open("TOVs/TOV_ap4_"*"$temp"*"_$z.csv", "a")
            #store for testing dataset
            dumping = open("TOVsTest/TOV_sly_"*"$temp"*"_$z.csv", "a")
            writedlm(dumping, DATA_matrix, ' ')
            close(dumping)         
        end
    end
end;

Λ = -0.0005326622105352159
j = 1
  5.145330 seconds (286.28 M allocations: 5.171 GiB, 8.19% gc time, 0.76% compilation time: 100% of which was recompilation)
j = 2
  5.416291 seconds (303.80 M allocations: 5.467 GiB, 8.26% gc time, 0.86% compilation time: 100% of which was recompilation)
j = 3
  5.475339 seconds (306.85 M allocations: 5.551 GiB, 7.84% gc time, 0.83% compilation time: 100% of which was recompilation)
j = 4
  5.098067 seconds (283.40 M allocations: 5.152 GiB, 8.14% gc time, 0.85% compilation time: 100% of which was recompilation)
j = 5
  5.961533 seconds (317.48 M allocations: 5.719 GiB, 8.19% gc time, 0.78% compilation time: 100% of which was recompilation)
j = 6
  3.729454 seconds (201.73 M allocations: 3.678 GiB, 8.49% gc time, 1.23% compilation time: 100% of which was recompilation)
j = 7
  5.153940 seconds (277.91 M allocations: 5.021 GiB, 8.53% gc time, 0.92% compilation time: 100% of which was recompilation)
j = 8
  6.387478 seconds (337.03 M allocations: 6.064 Gi

  5.688174 seconds (291.10 M allocations: 5.257 GiB, 7.65% gc time, 0.77% compilation time: 100% of which was recompilation)
j = 64
  5.114743 seconds (269.07 M allocations: 4.898 GiB, 7.58% gc time, 0.89% compilation time: 100% of which was recompilation)
j = 65
  3.502642 seconds (182.57 M allocations: 3.319 GiB, 7.60% gc time, 1.26% compilation time: 100% of which was recompilation)
j = 66
  5.083711 seconds (273.02 M allocations: 4.961 GiB, 7.51% gc time, 0.84% compilation time: 100% of which was recompilation)
j = 67
  4.559778 seconds (247.95 M allocations: 4.546 GiB, 7.38% gc time, 0.94% compilation time: 100% of which was recompilation)
j = 68
  4.738917 seconds (266.14 M allocations: 4.843 GiB, 7.41% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 69
  5.102556 seconds (285.26 M allocations: 5.147 GiB, 7.77% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 70
  4.901397 seconds (275.48 M allocations: 4.991 GiB, 7.79% gc time, 0.85% co

j = 25
  5.218760 seconds (262.85 M allocations: 4.820 GiB, 7.11% gc time, 0.90% compilation time: 100% of which was recompilation)
j = 26
  4.692223 seconds (255.91 M allocations: 4.674 GiB, 7.62% gc time, 0.94% compilation time: 100% of which was recompilation)
j = 27
  5.241760 seconds (264.14 M allocations: 4.814 GiB, 7.87% gc time, 0.79% compilation time: 100% of which was recompilation)
j = 28
  5.661257 seconds (287.10 M allocations: 5.196 GiB, 7.65% gc time, 0.81% compilation time: 100% of which was recompilation)
j = 29
  6.449824 seconds (317.54 M allocations: 5.758 GiB, 7.77% gc time, 0.78% compilation time: 100% of which was recompilation)
j = 30
  5.242204 seconds (248.73 M allocations: 4.569 GiB, 7.58% gc time, 0.91% compilation time: 100% of which was recompilation)
j = 31
  5.153834 seconds (260.12 M allocations: 4.759 GiB, 7.62% gc time, 0.98% compilation time: 100% of which was recompilation)
j = 32
  5.188892 seconds (263.18 M allocations: 4.816 GiB, 7.79% gc time, 0

j = 88
  5.657216 seconds (283.49 M allocations: 5.164 GiB, 8.11% gc time, 0.85% compilation time: 100% of which was recompilation)
j = 89
  5.857245 seconds (290.54 M allocations: 5.312 GiB, 7.92% gc time, 0.84% compilation time: 100% of which was recompilation)
j = 90
  5.499819 seconds (274.11 M allocations: 4.981 GiB, 7.85% gc time, 0.88% compilation time: 100% of which was recompilation)
j = 91
  5.032374 seconds (250.26 M allocations: 4.592 GiB, 7.65% gc time, 0.93% compilation time: 100% of which was recompilation)
j = 92
  5.128855 seconds (257.59 M allocations: 4.709 GiB, 8.11% gc time, 0.92% compilation time: 100% of which was recompilation)
j = 93
  5.500052 seconds (273.47 M allocations: 4.971 GiB, 8.12% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 94
  6.219682 seconds (311.53 M allocations: 5.628 GiB, 7.90% gc time, 0.78% compilation time: 100% of which was recompilation)
j = 95
  5.581035 seconds (279.79 M allocations: 5.115 GiB, 7.83% gc time, 0

  5.513727 seconds (275.78 M allocations: 5.026 GiB, 7.77% gc time, 0.88% compilation time: 100% of which was recompilation)
j = 51
  7.349633 seconds (371.15 M allocations: 6.662 GiB, 7.93% gc time, 0.68% compilation time: 100% of which was recompilation)
j = 52
  5.741329 seconds (287.11 M allocations: 5.223 GiB, 7.85% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 53
  5.684229 seconds (283.10 M allocations: 5.134 GiB, 8.02% gc time, 0.86% compilation time: 100% of which was recompilation)
j = 54
  6.021377 seconds (291.03 M allocations: 5.260 GiB, 7.80% gc time, 0.81% compilation time: 100% of which was recompilation)
j = 55
  6.066092 seconds (296.74 M allocations: 5.359 GiB, 8.11% gc time, 0.81% compilation time: 100% of which was recompilation)
j = 56
  5.669549 seconds (278.96 M allocations: 5.075 GiB, 7.84% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 57
  5.407045 seconds (270.85 M allocations: 4.913 GiB, 7.82% gc time, 0.88% co

...

j = 50
  7.176041 seconds (275.65 M allocations: 5.024 GiB, 7.32% gc time, 0.90% compilation time: 100% of which was recompilation)
j = 51
 10.142946 seconds (363.74 M allocations: 6.537 GiB, 7.73% gc time, 0.58% compilation time: 100% of which was recompilation)
j = 52
  8.531790 seconds (286.56 M allocations: 5.214 GiB, 7.77% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 53
  8.329827 seconds (278.76 M allocations: 5.057 GiB, 7.70% gc time, 1.12% compilation time: 100% of which was recompilation)
j = 54
  8.052335 seconds (289.57 M allocations: 5.238 GiB, 7.67% gc time, 0.86% compilation time: 100% of which was recompilation)
j = 55
  8.585020 seconds (296.03 M allocations: 5.348 GiB, 7.15% gc time, 1.03% compilation time: 100% of which was recompilation)
j = 56
  7.236811 seconds (274.44 M allocations: 4.995 GiB, 7.31% gc time, 1.03% compilation time: 100% of which was recompilation)
j = 57
  7.087185 seconds (270.98 M allocations: 4.916 GiB, 7.37% gc time, 0

  8.842600 seconds (328.27 M allocations: 5.930 GiB, 7.51% gc time, 0.76% compilation time: 100% of which was recompilation)
j = 13
  6.880450 seconds (259.21 M allocations: 4.779 GiB, 7.41% gc time, 0.99% compilation time: 100% of which was recompilation)
j = 14
  8.318829 seconds (304.61 M allocations: 5.511 GiB, 7.74% gc time, 0.80% compilation time: 100% of which was recompilation)
j = 15
  6.309943 seconds (229.84 M allocations: 4.215 GiB, 7.73% gc time, 0.98% compilation time: 100% of which was recompilation)
j = 16
  5.601655 seconds (199.39 M allocations: 3.632 GiB, 7.65% gc time, 1.13% compilation time: 100% of which was recompilation)
j = 17
  7.637802 seconds (248.30 M allocations: 4.554 GiB, 8.16% gc time, 0.90% compilation time: 100% of which was recompilation)
j = 18
  8.548973 seconds (279.69 M allocations: 5.097 GiB, 8.00% gc time, 0.76% compilation time: 100% of which was recompilation)
j = 19
  8.165001 seconds (271.48 M allocations: 4.940 GiB, 8.00% gc time, 0.89% co

  7.136019 seconds (276.80 M allocations: 5.068 GiB, 7.36% gc time, 0.93% compilation time: 100% of which was recompilation)
j = 76
  7.758388 seconds (293.26 M allocations: 5.322 GiB, 7.76% gc time, 0.81% compilation time: 100% of which was recompilation)
j = 77
  7.168895 seconds (278.21 M allocations: 5.047 GiB, 7.83% gc time, 0.88% compilation time: 100% of which was recompilation)
j = 78
  6.679963 seconds (257.85 M allocations: 4.706 GiB, 7.64% gc time, 0.93% compilation time: 100% of which was recompilation)
j = 79
  6.957615 seconds (269.14 M allocations: 4.904 GiB, 7.44% gc time, 0.93% compilation time: 100% of which was recompilation)
j = 80
  8.005097 seconds (308.91 M allocations: 5.596 GiB, 7.72% gc time, 0.78% compilation time: 100% of which was recompilation)
j = 81
  7.349218 seconds (286.14 M allocations: 5.190 GiB, 7.79% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 82
  7.914495 seconds (277.67 M allocations: 5.074 GiB, 7.91% gc time, 0.75% co

j = 37
  4.587615 seconds (207.97 M allocations: 3.778 GiB, 7.89% gc time, 1.02% compilation time: 100% of which was recompilation)
j = 38
  2.611725 seconds (128.52 M allocations: 2.331 GiB, 7.26% gc time, 2.22% compilation time: 100% of which was recompilation)
j = 39
  5.002211 seconds (273.14 M allocations: 4.994 GiB, 7.41% gc time, 0.87% compilation time: 100% of which was recompilation)
j = 40
  5.613597 seconds (292.98 M allocations: 5.311 GiB, 7.86% gc time, 0.73% compilation time: 100% of which was recompilation)
j = 41
  5.793533 seconds (304.11 M allocations: 5.505 GiB, 7.41% gc time, 0.77% compilation time: 100% of which was recompilation)
j = 42
  5.256475 seconds (268.55 M allocations: 4.886 GiB, 7.52% gc time, 0.82% compilation time: 100% of which was recompilation)
j = 43
  5.367603 seconds (265.76 M allocations: 4.833 GiB, 7.66% gc time, 0.80% compilation time: 100% of which was recompilation)
j = 44
  5.705677 seconds (274.33 M allocations: 4.960 GiB, 7.48% gc time, 0

j = 100
  5.800905 seconds (316.21 M allocations: 5.740 GiB, 7.35% gc time, 0.80% compilation time: 100% of which was recompilation)


## Observation procedure M-R

This section is dedicated to the creation of datasets including only M-R values. One can choose the number N of observation points on the M-R curve (e.g. we worked with N=15,20,30), and the number ns of Gaussian noise injections (e.g. we worked with ns=100,200,300).

In [ ]:
function DataGenerator(R_rand::AbstractVector{T}, M_rand::AbstractVector{T},
        DATA_matrix, iMax, σM, σR, TOT) where T
    #interpolate R-M curve
    M_array = Interpolations.deduplicate_knots!(DATA_matrix[:,2];move_knots = true);
    RMcurve = linear_interpolation(M_array[1:iMax],DATA_matrix[1:iMax,3])
    
    M_rand = rand(Uniform(M_array[1],M_array[iMax]),TOT)
    R_rand = RMcurve.(M_rand)
    #Shifting the data
    M_rand = rand.(Normal.(M_rand,σM))./Mnorm
    R_rand = rand.(Normal.(R_rand,σR))./Rnorm
    return R_rand, M_rand
end;

#functions to search for file in folder
searchdir(path,key) = filter(x->occursin(key,x), readdir(path));

In [ ]:
Λtemp= [-194, -150, -120, -95, -50, 0, 50, 95, 120, 194]
const σM = 0.1
const σR = 0.5
const Mnorm = 3.
const Rnorm = 20.
ns = 300 #100 #200 #300
TOT = 30 #Choose number of observation points 15, 20, 30

for i = 1:length(Λtemp)
    @show i
    
    tempΛ = Λtemp[i]
    listFile = searchdir("TOVs","_$tempΛ"*"_")
    
    for j = 1:length(listFile)
        #@show j
        #obtain remaining features
        eos = split(split(listFile[j], ".")[1],"_")[2]
        z = parse(Int,split(split(listFile[j], ".")[1],"_")[4])
        #extract TOV
        TOV_matrix = readdlm("TOVs/"*listFile[j], Float64)
        
        #extract random observation set ns times for each eos
        for h = 1:ns
            R_rand = zeros(0)
            M_rand = zeros(0)
            _, iMax = findmax(TOV_matrix[:,2])
            R_rand, M_rand = DataGenerator(R_rand, M_rand, TOV_matrix, iMax, σM, σR, TOT)
        
            dumping = open("dataset_30points_larger300.csv", "a");
            writedlm( dumping,  [eos tempΛ adjoint(matrix_ρ[z,:]) adjoint(matrix_cs[z,:]) adjoint(M_rand) adjoint(R_rand)], ' ')
            close(dumping)
        end
    end
end

## Observation procedure M-R-k2

This section is dedicated to the creation of datasets including M-R-k2 triplets. One can choose the number N of observation points on the M-R curve (e.g. we worked with N=15,20,30), and the number ns of Gaussian noise injections (e.g. we worked with ns=100,200,300).

Note: remember to search in correct folder (training and validation datasets in "TOVs", testing in "TOVsTest")

In [13]:
function DataGenerator(R_rand::AbstractVector{T}, M_rand::AbstractVector{T},
        k2_rand::AbstractVector{T}, DATA_matrix, iMax, TOT) where T
    #interpolate R-M curve
    M_array = Interpolations.deduplicate_knots!(DATA_matrix[:,2];move_knots = true);
    k2array = (3.0 / 2.0) .* DATA_matrix[:,4] .* DATA_matrix[:,2] .^5  ./DATA_matrix[:,3].^5/ KK^5
    RMcurve = linear_interpolation(M_array[1:iMax],DATA_matrix[1:iMax,3])
    k2Mcurve = linear_interpolation(M_array[1:iMax],k2array[1:iMax])
    
    M_rand = rand(Uniform(M_array[1],M_array[iMax]),TOT)
    R_rand = RMcurve.(M_rand)
    k2_rand = k2Mcurve.(M_rand)
    #Shifting the data
    M_rand = rand.(Normal.(M_rand,σM))./Mnorm
    R_rand = rand.(Normal.(R_rand,σR))./Rnorm
    k2_rand = rand.(Normal.(k2_rand,σk2))#renormalization done after
    return R_rand, M_rand, k2_rand
end;

#functions to search for file in folder
searchdir(path,key) = filter(x->occursin(key,x), readdir(path));

In [18]:
Λtemp= [-194, -150, -120, -95, -50, 0, 50, 95, 120, 194]
const σM = 0.1
const σR = 0.5
const σk2 = 0.05
const Mnorm = 3.
const Rnorm = 20.
ns = 100 #100 #200 #300
TOT = 30 #Choose number of observation points 15, 20, 30

for i = 1:length(Λtemp)
    @show i
    
    tempΛ = Λtemp[i]
    #listFile = searchdir("TOVs","_$tempΛ"*"_")
    listFile = searchdir("TOVsTest","_$tempΛ"*"_")
    
    for j = 1:length(listFile)
        #@show j
        #obtain remaining features
        eos = split(split(listFile[j], ".")[1],"_")[2]
        z = parse(Int,split(split(listFile[j], ".")[1],"_")[4])
        #extract TOV
        #TOV_matrix = readdlm("TOVs/"*listFile[j], Float64)
        TOV_matrix = readdlm("TOVsTest/"*listFile[j], Float64)
        
        #extract random observation set ns times for each eos
        for h = 1:ns
            R_rand = zeros(0)
            M_rand = zeros(0)
            k2_rand = zeros(0)
            _, iMax = findmax(TOV_matrix[:,2])
            R_rand, M_rand, k2_rand = DataGenerator(R_rand, M_rand, k2_rand, TOV_matrix, iMax, TOT)
        
            dumping = open("datasetk2_30points_Test_100.csv", "a");
            writedlm( dumping,  [eos tempΛ adjoint(matrix_ρ[z,:]) adjoint(matrix_cs[z,:]) adjoint(M_rand) adjoint(R_rand) adjoint(k2_rand)], ' ')
            close(dumping)
        end
    end
end

i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10


## Analysis of predicted EOSs

Let us now analyse the predicted EOS from our deep neural networks (done in a separate python notebook). Given and EOS, we solve the TOV and tidal deformability equations and we plot the results comparing them with the expected ones.

### Integrating the equations

We integrate here the TOV and tidal deformability equations.

Note: Load the appropriate functions.

In [ ]:
const ρt = 2.0 * n0 * m * ρu #mass density when transition happens
const ρfin = 12.0 * n0 * m * ρu
const pc = 200.0^4 * Kp * Pu #threshold corresponds to qcd scale
#define initial values for solving integration as global
const r0 = 10^-5
const a0 = 1.0
const f0 = 1.0
const h0 = 1.0
const H0 = a0 * r0^2
const β0 = 2. * a0 * r0
const rspan = (r0, 200);

In [ ]:
ϵ_fluid = zeros(0)
ϵ_prime(x::Any) = zeros(0)
DATA_matrix = Matrix{Float64}(undef,0,4)
Λ=0.

#choose which EOS to plot (EOS are saved in python notebook)
j = 100
N = 30
n = 10
EOS = readdlm("EOSj$j"*"N$N"*"n$n.txt", Float64)

coeff = 0.0052048215522052
matrix_ρ = EOS[:,3:9] .* coeff
matrix_cs = EOS[:,10:16]

for q = 2:11
    @show(q)

    z = q #It goes from 1 to 11 (1 is real data, other 10 predicted)
    temp = EOS[z,2] * 388. - 194.
    Λ = sign(temp)*(EOS[z,2] * 388. - 194.)^4 * Kp * Pu
    @show(Λ)

    if EOS[z,1] == 0
        baseline_eos = ap4
    else
        baseline_eos = sly
    end
    
    
    eos_matrix = Matrix{Float64}(undef,0,3) #define matrix to store complete eos
    eos_matrix = build(eos_matrix, baseline_eos, matrix_ρ, matrix_cs, z)

    #ϵ[p] before transition
    ϵ_fluid = linear_interpolation(eos_matrix[:,1],eos_matrix[:,2])
    #define interpolated dϵ[p]/dp
    ϵ_prime(x) = only(Interpolations.gradient(ϵ_fluid, x) );
        
    #INTEGRATOR
    P0 = 2.5 * 10^-5  #this will vary at each step of loop
    Pf = 0. #define it
    #Find what Pf is
    if Λ > 0. && eos_matrix[end,1] > pc
        Pf = eos_matrix[end,1] - Λ
    else
        Pf = eos_matrix[end,1]
    end

    #create empty matrix to store data in loop
    DATA_matrix = Matrix{Float64}(undef,0,4) 

    @time DATA_matrix = cycleTOV(DATA_matrix, P0, Pf)
    
    dumping = open("TOV_j$j"*"_N$N"*"_ns$n"*"_$z.csv", "a")
    writedlm(dumping, DATA_matrix, ' ')
    close(dumping)
    
end;

### Plots

#### Plotting curves

In [6]:
const Mnorm = 3.
const Rnorm = 20.
const σM = 0.1
const σR = 0.5
const σk2 = 0.05
k2max, k2min = 1.66007354312931, -1.57981785098554
#To compare with real curves, it's important to find the corresponding saved file in TOVsTest
#At the moment this is done manually, for j=100 and j=1210 this is:
realMR = readdlm("TOVsTest/TOV_ap4_-194_70.csv", Float64); #j=100
#realMR = readdlm("TOVsTest/TOV_sly_120_85.csv", Float64); #j=1210

In [ ]:
xvalue = [10,11,12,13]
xstring = [L"10",L"11",L"12",L"13"]
yvalue = [0.5,1.0,1.5,2.0]
ystring = [L"0.5",L"1.0", L"1.5", L"2.0"]

#REMEMBER to change labels accordingly
j = 100
N = 30
n = 10

EOS_data = readdlm("TOV_j$j"*"_N$N"*"_ns$n"*"_2.csv", Float64)

plotFig = plot(EOS_data[:,3],EOS_data[:,2],color=palette(:tab10)[1],label="",
    xlabel=L"R\ [\textrm{km}]",ylabel=L"M\ [M_\odot]",
    yticks=(yvalue, ystring), 
    xticks=(xvalue, xstring),
    xtickfont=font(12), ytickfont=font(12),framestyle=:box, dpi=300,
    legendfontsize=10,xguidefontsize=14,yguidefontsize=14,legend=:bottomleft)
for i=3:11
    EOS_data = readdlm("TOV_j$j"*"_N$N"*"_ns$n"*"_$i.csv", Float64)
    plot!(EOS_data[:,3],EOS_data[:,2],color=palette(:tab10)[i-1],label="")
end

plot!(realMR[:,3],realMR[:,2],color="black", linewidth=1.5,
    label=L"\textrm{real: AP4},\Lambda=-(194\ \textrm{MeV})^4", labelfont=font(12))

shift_data = readdlm("EOSj$j"*"N$N"*"n$n"*"real.txt", Float64)
scatter!(shift_data[31:60].* Rnorm,shift_data[1:30].*Mnorm, yerr = σM, xerr = σR,label="",
    color="grey",alpha=0.6)
#savefig(plotFig,"MR_N194.png");

In [ ]:
xvalue = [0.5,1.0,1.5,2.0]
xstring = [L"0.5",L"1.0", L"1.5", L"2.0"]
yvalue = [0.05,0.07,0.09,0.11]
ystring = [L"0.05",L"0.07", L"0.09", L"0.11"]

j = 100
N = 30
n = 10
EOS_data = readdlm("TOV_j$j"*"_N$N"*"_ns$n"*"_2.csv", Float64)

plotFig=plot(EOS_data[:,2],
    (3.0 / 2.0) .* EOS_data[:,4] .* EOS_data[:,2] .^5  ./EOS_data[:,3].^5/ KK^5,
    color=palette(:tab10)[1],label="",
    xlabel=L"M\ [M_\odot]",ylabel=L"k_2",
    yticks=(yvalue, ystring), 
    xticks=(xvalue, xstring),
    xtickfont=font(12), ytickfont=font(12),framestyle=:box, dpi=300,
    legendfontsize=10,xguidefontsize=14,yguidefontsize=14,legend=:topright)

for i=3:11
    EOS_data = readdlm("TOV_j$j"*"_N$N"*"_ns$n"*"_$i.csv", Float64)
    plot!(EOS_data[:,2],
    (3.0 / 2.0) .* EOS_data[:,4] .* EOS_data[:,2] .^5  ./EOS_data[:,3].^5/ KK^5,
    color=palette(:tab10)[i-1],label="")

end

plot!(realMR[:,2],
    (3.0 / 2.0) .* realMR[:,4] .* realMR[:,2] .^5  ./realMR[:,3].^5/ KK^5,
    color="black",
    linewidth=1.5,label=L"\textrm{real: AP4},\Lambda=-(194\ \textrm{MeV})^4", 
    labelfont=font(12))

#savefig(plotFig,"k2M_N194.png");

In [ ]:
xvalue = [0.5,1.0,1.5,2.0]
xstring = [L"0.5",L"1.0", L"1.5", L"2.0"]
yvalue = [-0.3,-0.2,-0.1,0,0.1]
ystring = [L"-0.3",L"-0.2", L"-0.1",L"0",L"0.1"]

k2Mcurve0 = linear_interpolation(realMR[:,2],(3.0 / 2.0) .* 
    realMR[:,4] .* realMR[:,2] .^5  ./realMR[:,3].^5/ KK^5)

j = 100
N = 30
n = 10
EOS_data = readdlm("TOV_j$j"*"_N$N"*"_ns$n"*"_2.csv", Float64)
_, iMax = findmax(EOS_data[:,2])
k2Mcurve = linear_interpolation(EOS_data[1:iMax,2],(3.0 / 2.0) .* 
        EOS_data[1:iMax,4] .* EOS_data[1:iMax,2].^5 ./EOS_data[1:iMax,3].^5/ KK^5)
Diff = (k2Mcurve(EOS_data[1:iMax,2]) .- 
    k2Mcurve0(EOS_data[1:iMax,2]))./k2Mcurve0(EOS_data[1:iMax,2])

plotFig=plot(EOS_data[1:iMax,2], Diff,
    color=palette(:tab10)[1],label="",
    xlabel=L"M\ [M_\odot]",ylabel=L"(k_2-k_{2,\textrm{real}})/k_{2,\textrm{real}}",
    yticks=(yvalue, ystring), 
    xticks=(xvalue, xstring),
    xtickfont=font(12), ytickfont=font(12),framestyle=:box, dpi=300,
    legendfontsize=10,xguidefontsize=14,yguidefontsize=14,legend=:topright,
    title=L"\textrm{AP4},\Lambda=-(194\ \textrm{MeV})^4")
for i=3:11
    EOS_data = readdlm("TOV_j$j"*"_N$N"*"_ns$n"*"_$i.csv", Float64)
    _, iMax = findmax(EOS_data[:,2])
    k = 1
    if realMR[1,2] > EOS_data[1,2]
        k = findpos(EOS_data,realMR[1,2],2)+1
    end
    k2Mcurve = linear_interpolation(EOS_data[k:iMax,2],(3.0 / 2.0) .* 
        EOS_data[k:iMax,4] .* EOS_data[k:iMax,2].^5 ./EOS_data[k:iMax,3].^5/ KK^5)
    Diff = (k2Mcurve(EOS_data[k:iMax,2]) .- 
        k2Mcurve0(EOS_data[k:iMax,2]))./k2Mcurve0(EOS_data[k:iMax,2]);
    plot!(EOS_data[k:iMax,2],Diff, color=palette(:tab10)[i-1],label="")
end

plotFig
#savefig(plotFig,"k2Mdiff_N194.png");

#### Reconstruct EOS

In [ ]:
ϵ_fluid = zeros(0)
ϵ_prime(x::Any) = zeros(0)

#choose which EOS to plot (EOS are saved in python notebook)
j = 1210
N = 30
n = 10
EOS = readdlm("EOSj$j"*"N$N"*"n$n.txt", Float64)

Λ=0. #define Λ outside of loop
coeff = 0.0052048215522052
matrix_ρ = EOS[:,3:9] .* coeff
matrix_cs = EOS[:,10:16]

for q = 1:11
    z = q

    temp = EOS[z,2] * 388. - 194.
    Λ = sign(temp)*(EOS[z,2] * 388. - 194.)^4 * Kp * Pu
    
    if EOS[z,1] == 0
        baseline_eos = ap4
    else
        baseline_eos = sly
    end
 
    eos_matrix = Matrix{Float64}(undef,0,3) #define matrix to store complete eos
    eos_matrix = build(eos_matrix, baseline_eos, matrix_ρ, matrix_cs, z)
        
    #ϵ[p] before transition
    ϵ_fluid = linear_interpolation(eos_matrix[:,1], eos_matrix[:,2])
    
    if Λ > 0 && eos_matrix[end,1] > pc
        Pf = eos_matrix[end,1] - Λ
    else
        Pf = eos_matrix[end,1]
    end

    k = findpos(eos_matrix,Pf,1)
    tosave = [eos_matrix[1:k-1,1] eos_interp.(eos_matrix[1:k-1,1])] 
    writedlm("EOS_j$j"*"N$N"*"n$n"*"z$z.csv",  tosave, ' ')
end;

In [ ]:
j = 100
N = 30
n = 10
EOS_plot = readdlm("EOS_j$j"*"N$N"*"n$n"*"z1.csv", Float64)
plotFig=plot(EOS_plot[:,1]./Pu,EOS_plot[:,2]./ρu, yaxis=:log10, xaxis=:log10, 
    legend=:bottomright,
    ylimits=(3*10.0^14,6.0*10.0^15),xlimits=(9.0*10.0^34,1.1*10.0^36),#j=100
    #ylimits=(9*10.0^14,7.0*10.0^15),xlimits=(3*10.0^35,1.5*10.0^36),#j=1210
    color="black", linewidth=1.5,label="",
    labelfont=font(12),xlabel="",
    ylabel="",
    yformatter=_->"", 
    xformatter=_->"",
    xtickfont=font(12), ytickfont=font(12),framestyle=:box, dpi=300)
for i = 2:11
    EOS_plot = readdlm("EOS_j$j"*"N$N"*"n$n"*"z$i.csv", Float64)
    plot!(EOS_plot[:,1]./Pu,EOS_plot[:,2]./ρu, yaxis=:log10, xaxis=:log10,
        color=palette(:tab10)[i-1],label="")
end
plotFig
#savefig(plotFig,"inset__120.png");
savefig(plotFig,"inset__N194.png");

In [ ]:
#img = load("inset_120.png")
img = load("inset_N194.png")

xvalue = [10.0^10,10.0^20, 10.0^30]
xstring = [L"10^{10}",L"10^{20}",L"10^{30}"]
yvalue = [10.0^5,10.0^10,10.0^15]
ystring = [L"10^{5}",L"10^{10}", L"10^{15}"]

j = 100
N = 30
n = 10
EOS_plot = readdlm("EOS_j$j"*"N$N"*"n$n"*"z1.csv", Float64)
plotFig=plot(EOS_plot[:,1]./Pu,EOS_plot[:,2]./ρu, yaxis=:log10, xaxis=:log10, 
    legend=:bottomright,
    #ylimits=(10^-5,10^-2),xlimits=(10^-10,0),
    color="black", linewidth=1.5,label=L"\textrm{real: SLy},\Lambda=(120\ \textrm{MeV})^4",
    labelfont=font(12),xlabel=L"p[\textrm{g\ cm^{-1} s^{-2}}]",
    ylabel=L"\epsilon[\textrm{g\ cm^{-3}}]",
    yticks=(yvalue, ystring), 
    xticks=(xvalue, xstring),
    xtickfont=font(12), ytickfont=font(12),framestyle=:box, dpi=300,
    xguidefontsize=14,yguidefontsize=14,legendfontsize=10)

for i=2:11
    EOS_plot = readdlm("EOS_j$j"*"N$N"*"n$n"*"z$i.csv", Float64)
    plot!(EOS_plot[:,1]./Pu,EOS_plot[:,2]./ρu, yaxis=:log10, xaxis=:log10,
        color=palette(:tab10)[i-1],label="")
end


plot!([0,1],[0,1],reverse(img, dims = 1), yflip = false, aspect_ratio = :none,
    inset = (1, bbox(0.02, 0.02,0.58, 0.49, :top, :left)),ticks = nothing,
    subplot = 2,
    #bg_inside = nothing
)
plotFig
#savefig(plotFig,"EOS_120.png");

In [ ]:
EOS_data = readdlm("EOSj0N30n100.txt", Float64)
N = 100
δρ = (minimum(EOS_data[:,9])-maximum(EOS_data[:,3]))/N
for l = 2:101 #cycle on interpolated curve 
    curveInt = linear_interpolation(EOS_data[l,3:9],EOS_data[l,10:end])
    ρ0 = maximum(EOS_data[:,3])
    data_matrix = Matrix{Float64}(undef,0,1)
    for i = 1:N+1 #cycle on ρ
        speed = curveInt(ρ0)
        data_matrix = vcat(data_matrix, [speed])
        ρ0 += δρ
    end
    dumping = open("IntCurve100.csv", "a")
    writedlm(dumping, transpose(data_matrix), ' ')
    close(dumping)
end

In [ ]:
#TO FIND MAX AND MIN FOR EACH ρ
IntCurve = readdlm("IntCurve100.csv", Float64)
min_values = zeros(0)
max_values = zeros(0)
ρ_values = zeros(0)
N=100
δρ = (minimum(EOS_data[:,9])-maximum(EOS_data[:,3]))/N
ρ0 = maximum(EOS_data[:,3])

for i=1:N+1
    speedmin = minimum(IntCurve[:,i])
    speedmax = maximum(IntCurve[:,i])
        
    ρ_values = vcat(ρ_values, [ρ0])
    min_values = vcat(min_values, [speedmin])
    max_values = vcat(max_values, [speedmax])
    ρ0 += δρ
end

In [ ]:
#TO FIND MEAN AND σ FOR EACH ρ
IntCurve = readdlm("IntCurve100.csv", Float64)
mean_values = zeros(0)
sigma_values = zeros(0)
ρ_values = zeros(0)
N=100
δρ = (minimum(EOS_data[:,9])-maximum(EOS_data[:,3]))/N
ρ0 = maximum(EOS_data[:,3])

for i=1:N+1
    mean_val = mean(IntCurve[:,i])
    sigma_val = std(IntCurve[:,i])
        
    ρ_values = vcat(ρ_values, [ρ0])
    mean_values = vcat(mean_values, [mean_val])
    sigma_values = vcat(sigma_values, [sigma_val])
    
    ρ0 += δρ
end

In [ ]:
xvalue = [0,0.2,0.4,0.6,0.8,1]
xstring = [L"0",L"0.2",L"0.4",L"0.6",L"0.8",L"1"]
yvalue = [0,0.2,0.4,0.6,0.8,1]
ystring = [L"0",L"0.2",L"0.4",L"0.6",L"0.8",L"1"]
plotFig=plot(EOS_data[1,3:9],EOS_data[1,10:end], color=:black,linewidth=1.5,framestyle=:box, 
    dpi=300,
    label=L"\textrm{real}",labelfont=font(16),xlabel=L"\rho/\rho_f",ylabel=L"c_s",
    yticks=(yvalue, ystring), 
    xticks=(xvalue, xstring),
    ylimits=(0,1),xlimits=(0,1),
    xtickfont=font(12), ytickfont=font(12),
    xguidefontsize=16,yguidefontsize=16,legendfontsize=11)
for i =2:101
    plot!(EOS_data[i,3:9],EOS_data[i,10:end], color=palette(:tab10)[1],label = "",
    alpha=0.2)
end
#plot!(ρ_values,min_values,color="grey",label="",style=:dash)
#plot!(ρ_values,max_values,color="grey",label="",style=:dash)
#plot!(ρ_values,mean_values,color=palette(:tab10)[1],label="")
plot!(ρ_values,mean_values.+2.0 .*sigma_values,color=palette(:tab10)[1],label="")
#plot!(ρ_values,mean_values.-2.0 .*sigma_values,color="grey",label="",style=:dash)
plot!(ρ_values, mean_values.-2.0 .*sigma_values, 
    fillrange = mean_values.+2.0 .*sigma_values, 
    fillalpha = 0.35, color=palette(:tab10)[1], label="")


In [ ]:
data_test = readdlm("data_test.txt", Float64)
data_hat = readdlm("data_hat.txt", Float64);

In [ ]:
line = [0.39,0.8]
xvalue = [0.4,0.5,0.6,0.7,0.8]
xstring = [L"0.4",L"0.5",L"0.6",L"0.7",L"0.8"]
yvalue = [0.4,0.5,0.6,0.7,0.8]
ystring = [L"0.4",L"0.5",L"0.6",L"0.7",L"0.8"]
plotFig=scatter(data_test[:,1],data_hat[:,1],yerr = 3.0.*data_hat[:,2], label="",
    ylimits=(0.39,0.8),xlimits=(0.39,0.8),color=:black,markersize=3,framestyle=:box, 
    dpi=300,xlabel=L"\langle c_s \rangle_\textrm{real}",
    ylabel=L"\langle c_s \rangle_\textrm{pred}",
    yticks=(yvalue, ystring), 
    xticks=(xvalue, xstring),
    xtickfont=font(12), ytickfont=font(12),
    xguidefontsize=16,yguidefontsize=16,legendfontsize=11)
plot!(line,line,color=:grey,label="",linestyle=:dash)
#savefig(plotFig,"c_sMean.png");